# [IAPR][iapr]: Lab 3 ‒  Classification


**Group ID:** xx

**Author 1 (sciper):** Student Name 1 (xxxxx)  
**Author 2 (sciper):** Student Name 2 (xxxxx)   
**Author 3 (sciper):** Student Name 3 (xxxxx)   

**Release date:** 23.04.2021  
**Due date:** 07.05.2021 


## Important notes

The lab assignments are designed to teach practical implementation of the topics presented during class well as preparation for the final project, which is a practical project which ties together the topics of the course. 

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook !** rerun the notebook from scratch `Kernel` > `Restart & Run All`


[iapr]: https://github.com/LTS5/iapr

## Imports

In [ ]:
import os
import gzip
import tarfile
import numpy as np

import scipy.io

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

# Model
from sklearn.neural_network import MLPClassifier

# Parameter tuning
from sklearn.model_selection import GridSearchCV

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Extract relevant data
We first need to extract the `lab-03-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [ ]:
data_base_path = 'data'
data_folder = 'lab-03-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)

---
## Part 1
In this part, we will study classification based on the data available in the Matlab file `classification.mat` that you will under `lab-03-data/part1`.
There are 3 data sets in this file, each one being a training set for a given class.
They are contained in variables `a`, `b` and `c`.

**Note**: we can load Matlab files using the [scipy.io] module.

[scipy.io]: https://docs.scipy.org/doc/scipy/reference/io.html

In [ ]:
data_part1_path = os.path.join(data_base_path, data_folder, 'part1', 'classification.mat')
matfile = scipy.io.loadmat(data_part1_path)
a = matfile['a']
b = matfile['b']
c = matfile['c']

print(a.shape, b.shape, c.shape)

### Check distribution of classes

In [ ]:
fig = plt.figure(1, figsize=(12, 5))

plt.scatter(a[:,0], a[:,1])
plt.scatter(b[:,0], b[:,1])
plt.scatter(c[:,0], c[:,1]) 
plt.show()

In [ ]:
fig = plt.figure(1, figsize=(12, 5))

plt.hist(b[:,1], bins=15)    
plt.show()

In [ ]:
fig = plt.figure(1, figsize=(12, 5))

plt.hist(c[:,1], bins=15)    
plt.show()

In [ ]:
import scipy.stats as st
def get_best_distribution(data):
    dist_names = ["norm", "exponweib", "pareto", "genextreme"]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)

        params[dist_name] = param
        # Applying the Kolmogorov-Smirnov test
        D, p = st.kstest(data, dist_name, args=param)
        print("p value for "+dist_name+" = "+str(p))
        dist_results.append((dist_name, p))

    # select the best fitted distribution
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    # store the name of the best fit and its p value

    print("Best fitting distribution: "+str(best_dist))
    print("Best p value: "+ str(best_p))
    print("Parameters for the best fit: "+ str(params[best_dist]))

    return best_dist, best_p, params[best_dist]

### 1.1 Bayes method (5 pts)
Using the Bayes method, give the analytical expression of the separation curves between those three classes.
Do reasonable hypotheses about the distributions of those classes and estimate the corresponding parameters based on the given training sets.
Draw those curves on a plot, together with the training data.
For simplicity reasons, round the estimated parameters to the closest integer value.

### Gaussian distribution

In [ ]:
def get_gaussian_dist(data):
    mean = np.rint(np.mean(data, axis=0))
    cov = np.rint(np.cov(data, rowvar=0))
    
    return mean, cov

In [ ]:
mean_a, cov_a = get_gaussian_dist(a)
mean_b, cov_b = get_gaussian_dist(b)
mean_c, cov_c = get_gaussian_dist(c)

In [ ]:
def get_discriminant(data, mean, cov, prior, nb_classes):
    fct = -0.5*(np.matmul(np.matmul(data.T, np.linalg.inv(cov)), data)) \
            + 0.5*(np.matmul(np.matmul(data.T, np.linalg.inv(cov)), mean)) \
            - 0.5*(np.matmul(np.matmul(mean.T, np.linalg.inv(cov)), mean)) \
            + 0.5*(np.matmul(np.matmul(mean.T, np.linalg.inv(cov)), data)) \
            + np.log(prior) \
            - np.log((2*np.pi)**(nb_classes/2)*np.linalg.det(cov)**(0.5))
        
    return fct

In [ ]:
nb_classes = 3
prior = 1/3 

In [ ]:
discriminant_a = np.array([get_discriminant(a[i], mean_a, cov_a, prior, nb_classes) for i in range(len(a))])
discriminant_b = np.array([get_discriminant(b[i], mean_b, cov_b, prior, nb_classes) for i in range(len(b))])
discriminant_c = np.array([get_discriminant(c[i], mean_c, cov_c, prior, nb_classes) for i in range(len(c))])

In [ ]:
discriminant_a.shape

Prior is the same for all classes as the number of samples per class is the same.

In [ ]:
# discriminant_a = get_discriminant(a.T, mean_a, cov_a, prior, nb_classes)
# discriminant_b = get_discriminant(b.T, mean_b, cov_b, prior, nb_classes)
# discriminant_c = get_discriminant(c.T, mean_c, cov_c, prior, nb_classes)

In [ ]:
decision_bd_ab = discriminant_a - discriminant_b
decision_bd_ac = discriminant_a - discriminant_c
decision_bd_bc = discriminant_b - discriminant_c

### 1.2 Mahalanobis distance (5 pts)
For classes `a` and `b`, give the expression of the Mahalanobis distance used to classify a point in class `a` or `b`, and verify the obtained classification, in comparison with the "complete" Bayes classification, for a few points of the plane.

In [ ]:
def mahalanobis_dist(point, mean, cov):
    dist_from_mean = point - mean
    distance = np.matmul(np.matmul(dist_from_mean, np.linalg.inv(cov)), dist_from_mean)
    
    return distance ** 0.5

In [ ]:
all_points_ab = np.concatenate((a, b))
all_labels_ab = np.array(['a']*len(a) + ['b']*len(b))


nb_points = 40
idx_rand = np.random.choice(all_points_ab.shape[0], nb_points)
rand_points = all_points_ab[idx_rand]
rand_labels = all_labels_ab[idx_rand]

predictions = []
nb_errors = 0
for i in range(rand_points.shape[0]):
    dist_a = mahalanobis_dist(rand_points[i], mean_a, cov_a)
    dist_b = mahalanobis_dist(rand_points[i], mean_b, cov_b)
    label = 'a' if dist_a < dist_b else 'b'
    if label != rand_labels[i]:
        nb_errors += 1
        

accuracy = (rand_labels.shape[0]-nb_errors)/rand_labels.shape[0] * 100
print(f"Accuracy for {nb_points} random points: {accuracy} %")
    

---

## Part 2
In this part, we aim to classify digits using the complete version of MNIST digits dataset.
The dataset consists of 60'000 training images and 10'000 test images of handwritten digits.
Each image has size 28x28, and has assigned a label from zero to nine, denoting the digits value.
Given this data, your task is to construct a Multilayer Perceptron (MLP) for supervised training and classification and evaluate it on the test images.

Download the MNIST dataset (all 4 files) from http://yann.lecun.com/exdb/mnist/ under `lab-03-data/part2`.
You can then use the script provided below to extract and load training and testing images in Python. 

**! Warning**: When the lab was created the official MNIST repo was down, if it is still the case please use https://github.com/mkolod/MNIST.

### 2.1 Dataset loading
Here we first declare the methods `extract_data` and `extract_labels` so that we can reuse them later in the code.
Then we extract both the data and corresponding labels, and plot randomly some images and corresponding labels of the training set.

In [ ]:
def extract_data(filename, image_shape, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(np.prod(image_shape) * image_number)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(image_number, image_shape[0], image_shape[1])
    return data


def extract_labels(filename, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * image_number)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return labels

In [ ]:
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000

data_part2_folder = os.path.join(data_base_path, data_folder, 'part2')

train_images_path = os.path.join(data_part2_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_part2_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_part2_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_part2_folder, 't10k-labels-idx1-ubyte.gz')

train_images = extract_data(train_images_path, image_shape, train_set_size)
test_images = extract_data(test_images_path, image_shape, test_set_size)
train_labels = extract_labels(train_labels_path, train_set_size)
test_labels = extract_labels(test_labels_path, test_set_size)

In [ ]:
prng = np.random.RandomState(seed=123456789)  # seed to always re-draw the same distribution
plt_ind = prng.randint(low=0, high=train_set_size, size=10)

fig, axes = plt.subplots(1, 10, figsize=(12, 3))
for ax, im, lb in zip(axes, train_images[plt_ind], train_labels[plt_ind]):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(lb)

### 2.2 MLP (10 pts)

To create an MLP you are free to choose any library.
In case you don't have any preferences, we encourage you to use the [scikit-learn] package; it is a simple, efficient and free tool for data analysis and machine learning.
In this [link][sklearn-example], you can find a basic example to see how to create and train an MLP using [scikit-learn].
Your network should have the following properties:
* Input `x`: 784-dimensional (i.e. 784 visible units representing the flattened 28x28 pixel images).
* 100 hidden units `h`.
* 10 output units `y`, i.e. the labels, with a value close to one in the i-th class representing a high probability of the input representing the digit `i`.

If you need additional examples you can borrow some code from image classification tutorials.
However, we recommend that you construct a minimal version of the network on your own to gain better insights.

[scikit-learn]: http://scikit-learn.org/stable/index.html
[sklearn-example]: http://scikit-learn.org/stable/modules/neural_networks_supervised.html

In [ ]:
flattened_train = train_images.reshape(train_images.shape[0], train_images.shape[1]*train_images.shape[2])
flattened_test = test_images.reshape(test_images.shape[0], test_images.shape[1]*test_images.shape[2])

In [ ]:
clf = MLPClassifier(solver='adam', hidden_layer_sizes=(100, 10), random_state=1)
clf.fit(flattened_train, train_labels)

In [ ]:
y_pred = clf.predict(flattened_test)
print(classification_report(test_labels, y_pred))

In [ ]:
accuracy_score(test_labels, y_pred)

## Grid search

In [ ]:
mlp = MLPClassifier(max_iter=500)
parameter_space = {
    'hidden_layer_sizes': [(100, 10)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001, 0.01, 0.05],
    'learning_rate': ['constant','adaptive'],
}
mlp = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=4)
mlp.fit(flattened_train, train_labels)

In [ ]:
print('Best parameters found:\n', mlp.best_params_)
means = mlp.cv_results_['mean_test_score']
stds = mlp.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, mlp.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
y_true, y_pred = test_labels, mlp.predict(X_test)
print(classification_report(y_true, y_pred))